In [1]:
import random
import time
import copy

# Import symbolic execution
from MT19937 import MT19937, MT19937_symbolic

# Import XorSolver
from XorSolver import XorSolver

## Consistency between MT19937 and MT19937_symbolic classes


In [2]:
# Init MT199937 symbolic
rng_sym = MT19937_symbolic()

# Init state: list of 624 32 bit numbers, as per MT19937
state = list(random.getstate()[1][:-1])

# Init MT19937 with state
rng = MT19937(state.copy())

t = time.time()

for _ in range(1000):
    
    # Get random number n
    n = rng()
    
    # Get symbolic representation of n
    n_sym = rng_sym()
    
    # Check symbolic execution is correct
    assert n == rng_sym.cast_num(state, n_sym)
    
print("Time taken: {}s".format(time.time() - t))

# > Time taken: 2.27s with cast_num
# > Time taken: 0.12s without cast_num

Time taken: 2.5949084758758545s


## MT19937 Cloning of Python's random library getrandbits(4) with precomputed matrix

Precomputed matrix only support nbits=[4,8,12,16,20,24,28,32]. Code below only uses only nbits=4 bits of MT19937 output

In [3]:
# Init python's getrandbits with 4 bits
nbits = 4
rng = lambda: random.getrandbits(nbits)

# Collecting data
eqns = []
n_test = []
for _ in range(624*32//nbits):
    # Get random number from rng and save for later testing
    n = rng()
    n_test.append(n)

t = time.time()

# Cloning MT19937 from data
rng_clone = MT19937(state_from_data = (n_test, nbits))

print("Time taken: {}s".format(time.time() - t))

# Test if cloning has been successful
for n in n_test:
    assert n == rng_clone() >> (32-nbits), "Clone failed!"
    
print("[*] Cloning successful!")

Time taken: 1.4259557723999023s
[*] Cloning successful!


## MT19937 Cloning of Python's random library getrandbits(32) using Python-only solver

Uses all 32 bits of MT19937

In [4]:
### Creating the equations to solve

# Init python's getrandbits with 32 bits
rng = lambda: random.getrandbits(32)

# Init MT199937 symbolic
rng_sym = MT19937_symbolic()

# Build system of equations
eqns = []
n_test = []
for _ in range(624):
    
    # Get random number from rng and save for later testing
    n = rng()
    n_test.append(n)
    
    # Split n into binary (A list of bools)
    eqn_rhs_list = rng_sym._int2bits(n)
    
    # Get symbolic representation of n
    eqn_lhs_list = rng_sym()
    
    # Add to eqns
    for lhs,rhs in zip(eqn_lhs_list, eqn_rhs_list):
        eqns.append([lhs,rhs])

In [5]:
### Using the python only solver XorSolver.solve

nvars = 624*32
eqns_copy = copy.deepcopy(eqns)

# Initialise solver with eqns
solver = XorSolver(eqns_copy, nvars)

t = time.time()

# Solve eqns. Takes aroung 100s to solve
# verbose=False to suppress output
solver.solve(verbose=False)

print("Time taken to solve: {}s".format(time.time() - t))

# Clone MT19937 with solver.eqns (fully solved by now)
rng_clone = MT19937(state_from_solved_eqns = solver.eqns)

# Test if cloning has been successful
for n in n_test:
    assert n == rng_clone(), "Clone failed!"
    
print("[*] Cloning successful!")

Time taken to solve: 112.22281908988953s
[*] Cloning successful!


## MT19937 Cloning of Python's random library getrandbits(16) using cryptominisat

Using Python wrapper of cryptominisat available on XorSolver. Uses only 16 bits of MT19937 output

In [6]:
### Creating the equations to solve

# Init python's getrandbits with 16 bits
nbits = 16
rng = lambda: random.getrandbits(nbits)

# Init MT199937 symbolic
rng_sym = MT19937_symbolic()

# Build system of equations
eqns = []
n_test = []
for _ in range(624*32//nbits):
    
    # Get random number from rng and save for later testing
    n = rng()
    n_test.append(n)
    
    # Split n into binary (A list of bools) and only take 8 bits
    eqn_rhs_list = rng_sym._int2bits(n, bits=nbits)
    
    # Get symbolic representation of n
    eqn_lhs_list = rng_sym()
    
    # Add to eqns
    for lhs,rhs in zip(eqn_lhs_list, eqn_rhs_list):
        eqns.append([lhs,rhs])

In [7]:
### Using XorSolver's cryptominisat wrapper

nvars = 624*32
eqns_copy = copy.deepcopy(eqns)

# Initialise solver with eqns
solver = XorSolver(eqns_copy, nvars)

t = time.time()

# Solve eqns, takes about 250s without m4ri
cryptominisat_path = r".\bin\cryptominisat5_win10_x64_no-m4ri.exe"
cnf_file = "test.cnf"
solver.cryptominisat_solve(cryptominisat_path, cnf_file, verbose=0) # Disable all output with verbose=0

print("Time taken to solve: {}s".format(time.time() - t))

# Clone MT19937 with solver.eqns (fully solved by now)
rng_clone = MT19937(state_from_solved_eqns = solver.eqns)

# Test if cloning has been successful
for n in n_test:
    assert n == rng_clone() >> (32-nbits), "Clone failed!"
    
print("[*] Cloning successful!")

Time taken to solve: 245.00420260429382s
[*] Cloning successful!


## Reversing MT19937 State with MT19937.reverse_states

A functionality in progress. There are still bugs with it. For now it seems to work sometimes. In the event that it fails, simply choose a different chunk of data and reverse the state from there.

In [80]:
# Initialise rng (random.getrandbits)
rng = lambda: random.getrandbits(32)

# Increment state of random object (random.getstate()) 10000 times
# Save output for testing later
n_test = []
shift = 10000
for _ in range(shift):
    n = rng()
    n_test.append(n)

# save next 624 numbers for cloning
data = []
for _ in range(624):
    n = rng()
    n_test.append(n)
    data.append(n)
    
# rng's state has incremented 10000 times before we got the data we need to clone
# We want to recover rng's original state by 
# cloning rng's state after it was incremented 10000 times and
# Reversing the state by 10000 steps

# Clone rng's current state
rng_clone = MT19937(state_from_data = (data, 32))

# Reverse state by 10000
rng_clone.reverse_states(shift)

# Test if cloning has been successful
# Does not work all the time
n_wrong = []
for idx, n in enumerate(n_test):
    if rng_clone() != n:
        n_wrong.append(idx)
if len(n_wrong) == 0:
    print("Reversing successful!")
else:
    print("Reversing failed. Predicted {}/{} numbers wrong".format(len(n_wrong), len(n_test)))

Reversing failed. Predicted 500/10624 numbers wrong


In [88]:
## If the above fails, forward rng_clone a few states and reverse

rng_clone = MT19937(state_from_data = (data, 32))

offset = random.randint(0, 624*10)
print('Offset used:', offset)
for _ in range(offset):
    rng_clone()

rng_clone.reverse_states(shift + offset)

# Test if cloning has worked.
# Might have to repeat a few times for it to work
n_wrong = []
for idx, n in enumerate(n_test):
    if rng_clone() != n:
        n_wrong.append(idx)
if len(n_wrong) == 0:
    print("Reversing successful!")
else:
    print("Reversing failed. Predicted {}/{} numbers wrong".format(len(n_wrong), len(n_test)))

Offset used: 2349
Reversing successful!
